In [ ]:
#import relevant libraries 
import pandas as pd
import plotly.express as px
import panel as pn
import hvplot.pandas
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta,date
from pandas import DataFrame

In [ ]:
## Get Market Data for S&P500 
#engage API keys by activating .env file for Alpaca Api
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

#function to read the api data for stock ticker
#returns dataframe of closing price and daily returns for a given ticker symbol
def get_ticker_data(api,ticker):
    #load in historical data for provided ticker
    ticker_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')

    #Clean Data
    #Select the column we need,  "adjusted close", and drop the others
    ticker_df = ticker_df['5. adjusted close']
    #Sort earliest to latest so that .pct_change() function works right.
    ticker_df.sort_index(inplace=True, ascending=True)

    #create a dataframe column for the daily returns (pct_change) values and concat with SP500 close
    returns_df = ticker_df.pct_change()
    all_returns_df = pd.concat([ticker_df, returns_df], axis="columns", join="inner")

    #Change column names to avoid confusion
    columns = [f"{ticker} Close",f"{ticker} Daily Returns"]
    all_returns_df.columns = columns

    # Drop nulls
    all_returns_df.dropna(inplace=True)

    #drop duplicates
    all_returns_df.drop_duplicates(inplace=True) 
    return all_returns_df

In [ ]:
# function to get the SMA of the returns
# return EMA signal values dataframe
def get_EMA(all_returns_df,ticker):
    # Grab just the `date` and `close` from the dataset
    ema_signals = all_returns_df.loc[:, [f"{ticker} 500 close"]].copy()

    # Set the short window and long windows
    short_window = 50
    long_window = 100

    # Generate the short and long exponential moving averages (50 and 100 days, respectively)
    ema_signals["EWM50"] = ema_signals[f"{ticker} 500 close"].ewm(span=short_window).mean()
    ema_signals["EWM100"] = ema_signals[f"{ticker} 500 close"].ewm(span=long_window).mean()
    ema_signals["Signal"] = 0.0

    # Generate the trading signal 0 or 1,
    # where 0 is when the EWM50 is under the EWM100, and
    # where 1 is when the EWM50 is higher (or crosses over) the SMA100
    ema_signals["Signal"][short_window:] = np.where(
    ema_signals["EWM50"][short_window:] > ema_signals["EWM100"][short_window:], 1.0, 0.0
    )
    # Calculate the points in time at which a position should be taken, 1 or -1
    ema_signals["Entry/Exit"] = ema_signals["Signal"].diff()

    ema_signals.head(10)# Grab just the `date` and `close` from the dataset
    ema_signals = all_returns_df.loc[:, [f"{ticker} 500 close"]].copy()

    # Set the short window and long windows
    short_window = 50
    long_window = 100

    # Generate the short and long exponential moving averages (50 and 100 days, respectively)
    ema_signals["EWM50"] = ema_signals[f"{ticker} 500 close"].ewm(span=short_window).mean()
    ema_signals["EWM100"] = ema_signals[f"{ticker} 500 close"].ewm(span=long_window).mean()
    ema_signals["Signal"] = 0.0

    # Generate the trading signal 0 or 1,
    # where 0 is when the EWM50 is under the EWM100, and
    # where 1 is when the EWM50 is higher (or crosses over) the SMA100
    ema_signals["Signal"][short_window:] = np.where(
    ema_signals["EWM50"][short_window:] > ema_signals["EWM100"][short_window:], 1.0, 0.0
    )
    # Calculate the points in time at which a position should be taken, 1 or -1
    ema_signals["EMA Entry/Exit"] = ema_signals["Signal"].diff()

    return ema_signal_df

In [ ]:
# function to get the Bolinger Bands of the returns
# return Bolinger Band signal values dataframe
def get_Bolinger(all_returns_df,ticker):
    

In [ ]:
# function to get the MACD Value of the returns
# return MACD signal values dataframe
def get_MACD(all_returns_df,ticker):

In [ ]:
#function to get all trading signals
#return data frame holding trading signals
def get_trading_signals(ema_signal_df,boligner_signal_df,macd_signal_df):
    trading_signals_df=pd.concat([ema_signal_df, bolinger_signal_df,mcad_signal_df],axis=1,    join="inner")
    trading_signals_df['Signal']=trading_signals_df['EMA Signal']+ trading_signals_df['Bolinger Signal']+ trading_signals_df['MACD Signal']
    trading_signals_df['Overall Entry/Exit']=trading_signals_df['Signal'].diff()
    return trading_signals_df